In [1]:
# for training usage, you might want to add your own here
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
from torchvision import transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.patheffects as PathEffects
# from centerLoss import CenterLoss
import torch.optim.lr_scheduler as lr_scheduler
import matplotlib.cm as cm
%matplotlib inline

find an available GPU device and set it

# 1. data loading

In [2]:
data_root = {
    "train":"train",
    "valid":"valid",
    "test":"test"
}

def train_tf(x):
   
    im_aug = transforms.Compose([
        transforms.Resize(120),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(96),
        transforms.ColorJitter(brightness=0.5, contrast=0.5, hue=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
    x = im_aug(x)
    return x

def test_tf(x):
   
    im_aug = transforms.Compose([
        transforms.Resize(120),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
    x = im_aug(x)
    return x

In [3]:
# TODO: load training and validation data
X1=ImageFolder(data_root["train"],transform=train_tf)  
X2=ImageFolder(data_root["valid"],transform=test_tf)  

## I mix train and valid only when I do finetune .
## when finetune ,X1 and X2 use transform=test_tf to keep it work.
## random seed does not work,but i still use it.
init_seed = 1
torch.manual_seed(init_seed)
torch.cuda.manual_seed(init_seed)
np.random.seed(init_seed) 

X=torch.utils.data.ConcatDataset([X1,X2])
X_train,X_valid = torch.utils.data.random_split(X, [160000, 20000])
train_dataloader= DataLoader(X_train,batch_size=64,shuffle=True)
valid_dataloader= DataLoader(X_valid,batch_size=64,shuffle=True)

# 2. build model

In [4]:
## for loss
class CrossEntropyLabelSmooth(nn.Module):
    def __init__(self, num_classes, epsilon):
        super(CrossEntropyLabelSmooth, self).__init__()
        self.num_classes = num_classes
        self.epsilon = epsilon
        self.logsoftmax = nn.LogSoftmax(dim=1)
    def forward(self, inputs, targets):
        log_probs = self.logsoftmax(inputs)
        targets = torch.zeros_like(log_probs).scatter_(1, targets.unsqueeze(1), 1)
        targets = (1 - self.epsilon) * targets + self.epsilon / self.num_classes
        loss = (-targets * log_probs).mean(0).sum()
        return loss
## for net
class ConvNet(torch.nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.extract = torch.nn.Sequential(
            torchvision.models.resnet152(pretrained=True).conv1,
            torchvision.models.resnet152(pretrained=True).bn1,
            torchvision.models.resnet152(pretrained=True).relu,
            torchvision.models.resnet152(pretrained=True).maxpool,
            torchvision.models.resnet152(pretrained=True).layer1,
            torchvision.models.resnet152(pretrained=True).layer2,
            torchvision.models.resnet152(pretrained=True).layer3,
            torchvision.models.resnet152(pretrained=True).layer4,
            torchvision.models.resnet152(pretrained=True).avgpool,   
        )
        self.feat = torch.nn.Sequential(
            torch.nn.Linear(2048, 256),
        )
        for index,net in enumerate(self.feat):
            if index==0 :
                torch.nn.init.xavier_uniform_(net.weight)      
        self.pred = torch.nn.Sequential(
            torch.nn.Linear(256, 10),
        )  
        for index,net in enumerate(self.pred):
            if index==0:
                torch.nn.init.xavier_uniform_(net.weight)
                
    def forward(self, x):
        x = self.extract(x)    
        x = x.view(-1, 2048)
        feat = self.feat(x)
        pred = F.log_softmax(self.pred(feat),1)
        return feat ,pred

# 3. choose optimizer and loss

In [5]:
model = ConvNet()
## load model param
checkpoint = torch.load("old_model/init1228.pkl")
model.load_state_dict(checkpoint['model'])

## only for finetune
for param in model.parameters():
    param.requires_grad = False
for param in model.feat.parameters():
    param.requires_grad = True
for param in model.pred.parameters():
    param.requires_grad = True
## 

model=model.cuda()
optimizer4nn = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), 
                               lr=1e-3, 
                               momentum=0.9, 
                               weight_decay=0.0005)    
nllloss = CrossEntropyLabelSmooth(10,0.1).cuda()
scheduler  = lr_scheduler.CosineAnnealingLR(optimizer4nn, T_max=5,eta_min=4e-08)

# 4. train network

In [6]:
# from tqdm import tqdm_notebook
# def train(train_loader, model, epoch):
#     print("Training Epoch: {}".format(epoch))
#     model.train()
#     train_precision=0.0
#     for step, (data, target) in enumerate(train_loader,1):
#         data = Variable(data).cuda()
#         target = Variable(target).cuda()
#         feat, pred = model(data)
#         _, pred_label = pred.max(dim=1)
#         loss=nllloss(pred, target)
#         optimizer4nn.zero_grad()
#         loss.backward()
#         optimizer4nn.step()
#         train_precision += float(torch.sum(pred_label== target)) 
#         if step % 350 == 0:
#             print("Epoch: {} step: {} loss:{}".format(epoch, step,loss))
            
# #     train_precision /= 90000.0
#     train_precision /= 160000.0
#     print("train accuracy: {}%".format(train_precision * 100))
#     return train_precision
            
# def test(test_loader, model, epoch):
#     print("Predicting Epoch: {}".format(epoch))
#     model.eval()
#     precision=0.0
#     for (data, target) in tqdm_notebook(test_loader):
#         data = Variable(data).cuda()
#         target = Variable(target).cuda()

#         feature, pred = model(data)
#         _, pred_label = pred.max(dim=1)
        
#         precision += float(torch.sum(pred_label == target))

# #     precision /=90000.0
#     precision /=20000.0
#     print("Validation accuracy: {}%".format(precision * 100))
#     return precision
# startepoch=0
# best_precision=0.8
# for epoch in range(startepoch,startepoch+60):
#     train_precision=train(train_dataloader, model, epoch)
#     precision=test(valid_dataloader, model, epoch)
    
#     scheduler.step()
#     if precision>best_precision:
#         best_precision=precision
#         state = {"model":model.state_dict(),"epoch":epoch,
#                  "valid_acc":precision,
#                  "train_acc":train_precision,
#             }
#         PATH ="old_model/init1228.pkl"
#         torch.save(state, PATH)

# 6. Evaluate your network

In [7]:
# for test usage - do not modify this
import pandas as pd
import numpy as np
import math
import os

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from more_itertools import chunked
from tqdm import tqdm_notebook

In [8]:
class TestData(ImageFolder):
    def __getitem__(self, idx):
        # drop labels
        return super(TestData, self).__getitem__(idx)[0]

In [9]:
## use transform
def test_tf(x):
    im_aug = transforms.Compose([
        transforms.Resize(120),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
    x = im_aug(x)
    return x

test_datafolder = TestData(data_root["test"], transform=test_tf)
classes = np.array(test_datafolder.classes)
imgs = [x for (x, _) in test_datafolder.imgs]
labels = classes[[x for (_, x) in test_datafolder.imgs]]

In [10]:
# load all test images from disk(SSD) into memory
# this is expected to take minutes
# this may fails due to system kill signal - just try it again later
# testdata = next(iter(DataLoader(test_datafolder, batch_size=len(test_datafolder))))

In [11]:
model = model.cuda()

## DO NOT USE batch_size=90000 because i do transform =test_tf
def evaluate(model, testdata, imgs, labels, batch_size=256):
    files_loader = [x for x in chunked(imgs, batch_size)]
    test_dataloader = DataLoader(testdata, batch_size=batch_size)
    groundtruth_loader = chunked(labels, batch_size)
    num_batches = len(files_loader)
    
    model.eval()
    
    columns = ["file", "prediction", "ground_truth"]
    data_iter = iter(zip(test_dataloader, files_loader, groundtruth_loader))
    df = pd.DataFrame(columns=columns)
    for _ in tqdm_notebook(range(num_batches)):
        X, filepath, Y = next(data_iter)
        X=X.cuda()
        with torch.no_grad():
            idx = torch.argmax(model(X)[1], 1).cpu()
        prediction = classes[idx]
        buffer = pd.DataFrame(zip(filepath, prediction, Y), columns=columns)
        df = df.append(buffer)

    return df

In [12]:
df = evaluate(model, test_datafolder, imgs, labels)
df

,file,prediction,ground_truth
0,test/airplane/cifar10-test-10.png,airplane,airplane
1,test/airplane/cifar10-test-1001.png,airplane,airplane
2,test/airplane/cifar10-test-1010.png,airplane,airplane
3,test/airplane/cifar10-test-1018.png,airplane,airplane
4,test/airplane/cifar10-test-1022.png,airplane,airplane
...,...,...,...
139,test/truck/n04520170_9909.png,truck,truck
140,test/truck/n04520170_9936.png,truck,truck
141,test/truck/n04520170_9942.png,truck,truck
142,test/truck/n04520170_9963.png,truck,truck


check your model performance

In [13]:
accu = np.sum(df["ground_truth"] == df["prediction"])/float(len(df)) * 100
print(f"Accuracy {accu:.3f}%")

Accuracy 91.251%


save the results

In [14]:
df.to_csv("test.csv", index=False)

restart your kernel to free GPU resource

In [15]:
exit()